In [ ]:
import websocket
import json
from pprint import pprint
import pandas as pd
import requests
from apscheduler.schedulers.blocking import BlockingScheduler
import time

URL = "wss://ws-feed.pro.coinbase.com"

#products = requests.get('https://api.pro.coinbase.com/products/').json()
#pairs = [entry['id'] for entry in products]

pairs = ['BTC-USD', 'LTC-USD', '']

trades = {pair: pd.DataFrame() for pair in pairs}
ohlcvs = {pair: pd.DataFrame() for pair in pairs}

scheduler = BlockingScheduler()

def job(pair):
    
    global ohlcvs
    global trades
    
    df = trades[pair]
    
    df = df[['time', 'price', 'size', 'trade_id']]
    new_index = pd.DatetimeIndex(df['time'])
    df['size'] = df['size'].apply(lambda x: float(x))
    df = df.set_index(new_index).drop(columns='time')
    vol = df['size'].resample('1Min').sum()
    df['price'] = df['price'].astype(float)
    ohlc1m = df['price'].resample('1Min').ohlc()
    ohlcvs[pair] = pd.concat([ohlc1m, vol], axis=1)
    
def on_message(ws, message):
    """Callback executed when a message comes.
    Positional argument:
    message -- The message itself (string)
    """
    msg = json.loads(message)
    pprint(msg)
    
    global trades
    
    for pair in trades:
        if msg['product_id'] == pair:
            trades[pair] = trades[pair].append(msg, ignore_index=True)
            
        try:
            scheduler.add_job(job(pair), 'interval', minutes=1)
            scheduler.start()
        except:
            pass
          
def on_open(socket):
    """Callback executed at socket opening.
    Keyword argument:
    socket -- The websocket itself
    """
    params = {
        "type": "subscribe",
        "channels": [{"name": "matches", "product_ids": pairs}]
    }
    socket.send(json.dumps(params))
    
def main():
    """Main function."""
    ws = websocket.WebSocketApp(URL, on_open=on_open, on_message=on_message)
    ws.run_forever()
    
if __name__ == '__main__':
    main()